## 房屋資料抓取

In [4]:
import requests
headers = {
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'   
}
res = requests.get('https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow=30&totalRows=2404&timestamp=1539754751297', headers = headers)

In [5]:
res

<Response [200]>

In [11]:
import pandas
pandas.DataFrame(res.json()['data']['house_list']).head(3)

,address,area,browsenum,cartmodel,carttype,community_link,community_name,delivery,down_price_percent,fci_pai,...,saletype,section_name,shape_name,showhouseage,showprice,tag,title,type,unit_price,unitprice
0,忠孝東路五段31巷,15.16,382,,,/newCommunity-index.html?cid=50519,無爭,,NaN,,...,0,信義區,電梯大樓,10年,"1,790",[],市府極美無爭精品,2,118.07萬/坪,118.07
1,忠孝東路五段743巷,32.56,606,,,/newCommunity-index.html?cid=93550,達麗信義,,NaN,,...,0,信義區,電梯大樓,3年,"2,600","[含車位, 有陽台]",指名承辦人小鍾：本棟最低總價！精緻裝潢！,2,114.77萬/坪,114.77
2,松山路,20.68,720,,,/newCommunity-index.html?cid=6751,永春大樓,,NaN,,...,0,信義區,電梯大樓,34年,"1,220",[有陽台],專售永春大樓-住商住匠團隊,2,58.99萬/坪,58.99


In [14]:
import time
houseurl = 'https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow={}&totalRows=2404&timestamp=1539754751297'

houseary = []
for i in range(0,int(2404 / 30)+ 1 ):
    res = requests.get(houseurl.format(i*30), headers = headers)
    df = pandas.DataFrame(res.json()['data']['house_list'])
    houseary.append(df)
    time.sleep(1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [15]:
housedf = pandas.concat(houseary)

In [17]:
housedf.count()

address                 2403
area                    2405
browsenum               2405
build_purpose              2
call_num                   2
cartmodel               2403
carttype                2403
community_link          2403
community_name          2403
company                    2
delivery                2403
down_price_percent        35
fci_pai                 2403
floor                   2403
has_carport             2403
houseage                2405
houseid                 2405
is_carport              2403
is_combine              2403
is_down_price           2403
is_full                    2
is_hurry_price          2403
is_newhouse                2
is_oversea              2403
is_pro_advertisement       2
is_video                2403
isnew                   2403
isvip                   2403
kind                    2403
kind_name               2403
label                      2
mainarea                2403
nick_name               2403
original_price            35
phonenum      

In [18]:
housedf.to_excel('house591.xlsx')

## 房屋資料清理

In [47]:
import pandas
housedf  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/chinalife/master/data/house591.xlsx')

In [48]:
housedf.iloc[2]

address                                                            忠孝東路五段
area                                                                41.17
browsenum                                                             180
build_purpose                                                         NaN
call_num                                                              NaN
cartmodel                                                             NaN
carttype                                                              NaN
community_link                          /newCommunity-index.html?cid=6305
community_name                                                       春秋大廈
company                                                               NaN
delivery                                                              NaN
down_price_percent                                                    NaN
fci_pai                                                               NaN
floor                                 

In [49]:
housedf = housedf[['area', 'room','floor','price','houseage']]

In [50]:
housedf[['bedroom', 'livingroom', 'bathroom']] = housedf['room'].str.extract('(\d+)房(\d+)廳(\d+)衛')

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [51]:
housedf[['current_floor', 'total_floor']] = housedf['floor'].str.extract('(\d+)F/(\d+)F')

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [52]:
del housedf['room']
del housedf['floor']

In [53]:
housedf.dropna(inplace=True)

In [54]:
housedf.head()

,area,price,houseage,bedroom,livingroom,bathroom,current_floor,total_floor
2,41.17,3988,35,4,2,2,8,14
3,15.76,1180,44,2,1,1,3,4
6,106.74,11500,4,4,2,3,7,12
7,15.39,1520,19,1,1,1,4,14
8,53.39,3850,29,2,2,2,8,15


In [55]:
housedf.columns

Index(['area', 'price', 'houseage', 'bedroom', 'livingroom', 'bathroom',
       'current_floor', 'total_floor'],
      dtype='object')

In [59]:
Y = housedf['price'].values
X = housedf[['area', 'bedroom', 'livingroom', 'bathroom', 'current_floor',
       'total_floor', 'houseage']]

In [60]:
X['total_floor'] = X['total_floor'].astype(float)
X['bedroom'] = X['bedroom'].astype(float)
X['livingroom'] = X['livingroom'].astype(float)
X['bathroom'] = X['bathroom'].astype(float)
X['current_floor'] = X['current_floor'].astype(float)

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

## 資料分析

In [61]:
import statsmodels.api as sm
X2  = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     2502.
Date:                Wed, 17 Oct 2018   Prob (F-statistic):               0.00
Time:                        14:18:00   Log-Likelihood:                -18696.
No. Observations:                2177   AIC:                         3.741e+04
Df Residuals:                    2169   BIC:                         3.745e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1096.2896    137.992     -7.945

In [62]:
predictorcols = X.columns

In [64]:
import itertools
AICs = {}
for k in range(1,len(predictorcols)+1):
    for variables in itertools.combinations(predictorcols, k):
        predictors = X[list(variables)]
        predictors2 = sm.add_constant(predictors)
        est = sm.OLS(Y, predictors2)
        res = est.fit()
        AICs[variables] = res.aic

In [70]:
from collections import Counter
c = Counter(AICs)
c.most_common()[::-1][0:3]

[(('area', 'bedroom', 'bathroom', 'current_floor', 'total_floor', 'houseage'),
  37407.03375336106),
 (('area',
   'bedroom',
   'livingroom',
   'bathroom',
   'current_floor',
   'total_floor'),
  37407.10172648925),
 (('area', 'bedroom', 'bathroom', 'current_floor', 'total_floor'),
  37407.16576925829)]

In [63]:
import itertools
for variables in itertools.combinations(predictorcols, 4):
    print(variables)

('area', 'bedroom', 'livingroom', 'bathroom')
('area', 'bedroom', 'livingroom', 'current_floor')
('area', 'bedroom', 'livingroom', 'total_floor')
('area', 'bedroom', 'livingroom', 'houseage')
('area', 'bedroom', 'bathroom', 'current_floor')
('area', 'bedroom', 'bathroom', 'total_floor')
('area', 'bedroom', 'bathroom', 'houseage')
('area', 'bedroom', 'current_floor', 'total_floor')
('area', 'bedroom', 'current_floor', 'houseage')
('area', 'bedroom', 'total_floor', 'houseage')
('area', 'livingroom', 'bathroom', 'current_floor')
('area', 'livingroom', 'bathroom', 'total_floor')
('area', 'livingroom', 'bathroom', 'houseage')
('area', 'livingroom', 'current_floor', 'total_floor')
('area', 'livingroom', 'current_floor', 'houseage')
('area', 'livingroom', 'total_floor', 'houseage')
('area', 'bathroom', 'current_floor', 'total_floor')
('area', 'bathroom', 'current_floor', 'houseage')
('area', 'bathroom', 'total_floor', 'houseage')
('area', 'current_floor', 'total_floor', 'houseage')
('bedroom'

## 連線資料庫

In [71]:
import sqlite3 as lite
con = lite.connect('test.sqlite')
cur = con.cursor()
cur.execute('SELECT SQLITE_VERSION()')
data = cur.fetchone()
con.close()

In [72]:
data

('3.19.3',)

In [73]:
import sqlite3 as lite
with lite.connect('test.sqlite') as con:
    cur = con.cursor()
    cur.execute('SELECT SQLITE_VERSION()')
    data = cur.fetchone()
#con.close()

In [74]:
data

('3.19.3',)

In [75]:
import sqlite3 as lite 
with lite.connect("test.sqlite") as con: 
    cur = con.cursor() 
    cur.execute("DROP TABLE IF EXISTS PhoneAddress") 
    cur.execute("""CREATE TABLE PhoneAddress(
                    phone CHAR(10) PRIMARY KEY, 
                    address TEXT, 
                    name TEXT unique, 
                    age INT NOT NULL)""") 
    cur.execute("INSERT INTO PhoneAddress VALUES('0912173381','United State','Jhon Doe',53)") 
    cur.execute("INSERT INTO PhoneAddress VALUES('0928375018','Tokyo Japan','MuMu Cat',6)") 
    cur.execute("INSERT INTO PhoneAddress VALUES('0957209108','Taipei','Richard',29)") 


In [77]:
import sqlite3 as lite 
with lite.connect("test.sqlite") as con: 
    cur = con.cursor() 
    cur.execute("SELECT phone, address from PhoneAddress")
    data = cur.fetchall()

In [78]:
data

[('0912173381', 'United State'),
 ('0928375018', 'Tokyo Japan'),
 ('0957209108', 'Taipei')]

## Python and SQLServer


In [79]:
import sqlite3 as lite 
with lite.connect("test.sqlite") as con: 
    cur = con.cursor() 
    cur.execute("SELECT phone, address from PhoneAddress")
    data1 = cur.fetchone()
    data2 = cur.fetchall()

In [80]:
data1

('0912173381', 'United State')

In [81]:
data2

[('0928375018', 'Tokyo Japan'), ('0957209108', 'Taipei')]

In [82]:
import sqlite3 as lite
import pandas
employee = [{'name':'Mary', 'age':23 , 'gender': 'F'},{'name':'John', 'age':33 , 'gender': 'M'}]
df = pandas.DataFrame(employee)
df

,age,gender,name
0,23,F,Mary
1,33,M,John


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
age       2 non-null int64
gender    2 non-null object
name      2 non-null object
dtypes: int64(1), object(2)
memory usage: 128.0+ bytes


In [85]:
with lite.connect('test.sqlite') as db:
    df.to_sql(name='employee', index=False, con=db, if_exists='replace')


## 儲存實例

In [86]:
import pandas
housedf  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/chinalife/master/data/house591.xlsx')

In [88]:
housedf = housedf[['area', 'room','floor','price','houseage']]

In [90]:
housedf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2405 entries, 0 to 2
Data columns (total 5 columns):
area        2405 non-null float64
room        2405 non-null object
floor       2402 non-null object
price       2405 non-null int64
houseage    2405 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 112.7+ KB


In [91]:
with lite.connect('house.sqlite') as con:
    housedf.to_sql('house591', con = con)

In [93]:
with lite.connect('house.sqlite') as con:
    df = pandas.read_sql_query('select * from house591 limit 3', con = con)

In [94]:
df

,index,area,room,floor,price,houseage
0,0,75.00,3~4房,None,9000,0
1,1,46.42,開放式格局,9F/13F,4380,12
2,2,41.17,4房2廳2衛,8F/14F,3988,35


In [96]:
with lite.connect('house.sqlite') as con:
    df = pandas.read_sql_query('select avg(price) from house591', con = con)

In [97]:
df

,avg(price)
0,3706.310187


In [98]:
with lite.connect('house.sqlite') as con:
    df = pandas.read_sql_query('select avg(price/area) from house591', con = con)

In [99]:
df

,avg(price/area)
0,87.765449


## SQLAlchemy and SQLServer 
- https://docs.sqlalchemy.org/en/latest/core/engines.html

## 三大法人日報
- http://www.twse.com.tw/zh/page/trading/fund/T86.html

In [100]:
import requests
res = requests.get('http://www.twse.com.tw/fund/T86?response=json&date=20181016&selectType=ALL&_=1539761083842')

In [103]:
import pandas
df = pandas.DataFrame(res.json()['data'])

In [105]:
df.columns = res.json()['fields']

In [107]:
with lite.connect('test.sqlite') as con:
    df.to_sql('bigthree', con = con)

## 抓取公開資訊觀測站歷年變更登記

In [113]:
import requests
from bs4 import BeautifulSoup
payload = {
'encodeURIComponent': '1',
'step': '1',
'firstin': '1',
'off': '1', 
'queryName': 'co_id',
'inpuType': 'co_id',
'TYPEK': 'all',
'co_id': '2823'    
}
res  = requests.post('http://mops.twse.com.tw/mops/web/ajax_t05st05', data = payload)
soup = BeautifulSoup(res.text, 'lxml')

In [118]:
payload2 = {}
for item in soup.select_one('form').select('input[type=hidden]'):
    if item.get('value'):
        payload2[item.get('name')] = item.get('value')
payload2

{'TYPEK': 'sii', 'co_id': '2823', 'firstin': 'true', 'off': '1', 'step': '2'}

In [119]:
payload2['year'] = '1994'
payload2['month'] = '10'

In [120]:
res = requests.post('http://mops.twse.com.tw/mops/web/ajax_t05st05', data = payload2)
res

<Response [200]>

In [123]:
dfs = pandas.read_html(res.text)
dfs[1]

,0,1,2,3,4
0,是否係申報首次公開發行之輸入：,NaN,NaN,NaN,NaN
1,每 股 面 額,10.0000,變 更 公 司 執 照 時 間,83年,10月
2,核定股本股數(股),250000000,核定股本金額(元),2500000000,NaN
3,實收股本股數 (股),212875000,實收股本金額(元),2128750000,NaN
4,股本來源(元),1.創 立 時 資 本,20000000,2.現金增資,765600000
5,3.資本公積轉增資,250275000,4.盈餘轉增資,1064875000,NaN
6,5.證期局核准資本公積之日期,NaN,6.證期局核准盈餘轉增資之日期,NaN,NaN
7,7.合併增資(元),0,8.減資(元),0,NaN
8,9.證期局核准合併增資之日期,NaN,10.證期局核准減資之日期,NaN,NaN
9,11.其他,員工紅利轉增資:28000000,NaN,NaN,NaN


## 取得盤後交易資訊

## Rate Limiting
- https://www.nginx.com/blog/rate-limiting-nginx/

In [126]:
import requests
import time
for i in range(1,10):
    res = requests.get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date=&type=&_=1539763053868')
    time.sleep(3)
    print(res)
#res.json()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


## Proxy Rotate

In [135]:
import requests
headers = {
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',  
 'origin': 'https://www.whatismyip.com',
 'referer': 'https://www.whatismyip.com/'   
}

proxies = {
  'http': 'http://196.17.26.204:8800',
  'https': 'http://196.17.26.204:8800',
}
res = requests.get('https://ipv4.whatismyip.com/', headers = headers, proxies = proxies)
res.text

'196.17.26.204'